In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

print("Path to model files:", path)

100%|██████████| 11.1G/11.1G [10:28<00:00, 19.0MB/s]
Extracting model files...


Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [3]:
!pip install -q torch langchain bitsandbytes accelerate transformers sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 126.3 kB/s eta 0:00:00


In [4]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.0 MB/s eta 0:00:00


In [5]:
import os
import torch
import transformers

from glob import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser

import warnings
warnings.simplefilter('ignore')

In [6]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    print('Using GPU: ', torch.cuda.get_device_name(0))
    print('Memory Usage: ')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

else:
    print("GPU is not available.")

GPU is available.
Using GPU:  NVIDIA A100-SXM4-40GB
Memory Usage: 
Allocated:  0.0 GB
Cached:  0.0 GB


In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

In [8]:
model_id = "/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1"
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,do_sample=True,device_map = "auto")

tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
device = "cuda" # the device to load the model onto

prompt = "Tell me about Tax Exemption of Religious institutions."

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)[0]

print(decoded.replace('\\n', '\n'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Tell me about Tax Exemption of Religious institutions.

Tax Exemption of Religious Institutions in India:

The Constitution of India provides for the exemption of religious institutions and places of worship from the payment of taxes, subject to certain conditions. The tax exemption is provided under Article 30(1) of the Constitution, which guarantees freedom of religion and certain rights relating to religious practices.

Eligibility for Tax Exemption:

To be eligible for tax exemption, a religious institution or place of worship must be founded and administered primarily for religious purposes. It must not be received or controlled by the state or any government, local authority, or any religious authority outside India.

Exemptions:

Under Section 11 of the Income Tax Act, 1961, religious institutions and places of worship are exempt from payment of corporate income tax, provided they meet the eligibility criteria. Additionally, donations made to religious institutions and place

In [22]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.8 MB/s eta 0:00:00


In [23]:
paper_paths = glob("/content/ML.pdf")
pages = []

# Initialize the progress bar
progress_bar = tqdm(total=len(paper_paths), desc="Processing PDFs")

for path in paper_paths:
    try:
        loader = PyPDFLoader(path)
        doc = loader.load()

        # Chunk text
        text_splitter = CharacterTextSplitter(chunk_size=500,
                                              chunk_overlap=0)
        chunked_documents = text_splitter.split_documents(doc)

        pages.extend(chunked_documents)
    except Exception as e:
        print(f'Skipping {path} due to error: {e}')

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

Processing PDFs:   0%|          | 0/1 [00:00<?, ?it/s]

# **RAG Using LangChain**

In [24]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [25]:
prompt_template = """
Instruction: Answer the question based on the following context:
{context}

Question:
{question}
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [26]:
# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [27]:
# Load chunked documents into the FAISS index
db = FAISS.from_documents(
    pages,
    HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
)

# **RAG Creation**

In [28]:
# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [29]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [30]:
query = "Tell me about Data Science problems and Machine Learning."
response = rag_chain.invoke(query)

In [31]:
print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: Tell me about Data Science problems and Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
3 
 
 
 
 
 
• Tasks performed by Human Beings:  There are lots of tasks performed on day to day basis by 
human beings, but the main concer n is that to perform the tasks perfectly and to perform under 
well-defined program. Examples: Cooking, Driving, Speech Recognition.  
• Tasks beyond Human Capabilities:  Another category of tasks includes which can be done by 
machine learning in effective mann er is analysis of large and complex data sets like Remote 
Sensing, Weather forecasting, Ecommerce, Web search etc. With large amounts of data, it 
becomes really complex for human beings to predict meaningful data.  
 
Machine learning has pro

In [32]:
query = "What is unsupervised Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is unsupervised Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
7 
 
 
 
 
 
[4]. Supervised learning tasks can be further categorized as classification tasks and regression tasks. In 
case of classification, the output label s are discrete whereas they are continuous in case of regression . 
5.2. Unsupervised Learning  
The unsupervised learning approach is all about  recogniz ing unidentified  existing patterns from the data 
in order to  derive rules from them. This technique is appropriate  in a situation when the categories of 
data are unknown . Here, the training data is not labeled.  Unsupervised learning is regarded as a statistic 
based approach for learning and thus refers to the problem of finding hidden struc

In [33]:
query = "What is supervised Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is supervised Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
7 
 
 
 
 
 
[4]. Supervised learning tasks can be further categorized as classification tasks and regression tasks. In 
case of classification, the output label s are discrete whereas they are continuous in case of regression . 
5.2. Unsupervised Learning  
The unsupervised learning approach is all about  recogniz ing unidentified  existing patterns from the data 
in order to  derive rules from them. This technique is appropriate  in a situation when the categories of 
data are unknown . Here, the training data is not labeled.  Unsupervised learning is regarded as a statistic 
based approach for learning and thus refers to the problem of finding hidden structu

In [34]:
query = "What is self supervised Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is self supervised Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
7 
 
 
 
 
 
[4]. Supervised learning tasks can be further categorized as classification tasks and regression tasks. In 
case of classification, the output label s are discrete whereas they are continuous in case of regression . 
5.2. Unsupervised Learning  
The unsupervised learning approach is all about  recogniz ing unidentified  existing patterns from the data 
in order to  derive rules from them. This technique is appropriate  in a situation when the categories of 
data are unknown . Here, the training data is not labeled.  Unsupervised learning is regarded as a statistic 
based approach for learning and thus refers to the problem of finding hidden st

In [35]:
query = "What is Hybrid Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is Hybrid Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine learni

In [36]:
query = "What is deep Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is deep Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine learning and tre

In [37]:
query = "What is Reinforcement Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is Reinforcement Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
4 
 
 
 
 
 
• Regression Problem - Regression algorithms are used to deal with problems with continuous and 
numeric output. These are usually used for problems that deal with questions like, ‘how much’ 
or ‘how many’.  
• Clustering Problem - Clustering falls under the category of unsupervised l earning algorithms. 
These algorithms try to learn structures within the data and attempt to make clusters based on 
the similarity in the structure of data. The different classes or clusters are then labeled. The 
algorithm, when trained, puts new unseen dat a in one of the clusters.  
• Reinforcement Problem -Reinforcement algorithms are used when a decision is t

In [38]:
query = "What is an Artificial Neural Network."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is an Artificial Neural Networks.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
4 
 
 
 
 
 
• Regression Problem - Regression algorithms are used to deal with problems with continuous and 
numeric output. These are usually used for problems that deal with questions like, ‘how much’ 
or ‘how many’.  
• Clustering Problem - Clustering falls under the category of unsupervised l earning algorithms. 
These algorithms try to learn structures within the data and attempt to make clusters based on 
the similarity in the structure of data. The different classes or clusters are then labeled. The 
algorithm, when trained, puts new unseen dat a in one of the clusters.  
• Reinforcement Problem -Reinforcement algorithms are used when a decision is to

In [39]:
query = "What is ReLU in Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is ReLU in Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
2 
 
 
 
 
 
Machine learning is a multi -disciplinary field having  a wide -range of research domains reinforcing 
its existence.  These are as shown in the following figure 1.  
The simulation of ML models is significantly related to  Computational Statistics whose main aim is 
to focus on making predictions  via computers. It is also co -related to M athematical Optimization which 
relates models , applications and frameworks to the field of statistics.  
Real world problems have high complexity which m ake them excellent  candidates  for application of 
ML. Machine learning can be applied to various areas of computing to design and program explicit 
algorithms

In [40]:
query = "What is a Transformer in AI."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is a Transformer in AI.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine learning a

In [41]:
query = "What is a LSTM in Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: What is a LSTM Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine learni

In [42]:
query = "What is a GRU in Machine Learning."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is a GRU in Machine Learning.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine lear

In [44]:
query = "What is a Large Kanguage Model."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is a Large Kanguage Model.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine learnin

In [45]:
query = "What is Low Rank Adaptation in AI."
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: What is Low Rank Adaptation in AI.

Instruction: Answer the question based on the following context:
[Document(page_content='Second National Conference on Computational Intelligence (NCCI 2018) 
IOP Conf. Series: Journal of Physics: Conf. Series 1142  (2018) 012012 IOP Publishing 
doi:10.1088/1742-6596/1142/1/012012 
5 
 
 
 
 
 
recognizing human faces in photos.  
2015  Amazon proposed its own Machine Learning Platform. Microsoft created “Distributed Machine 
Learning Toolkit” for efficient distribution of machine learning problems to multiple computers to 
work parallel to find a solution  [10,11 ]. Elon Musk and Sam Altman, created a non -profit 
organization - OoeaAI, with the objectiv e of using Artificial Intelligence to serve human beings.  
2016  Google proposed DeepMind which is regarded as the most complex Board Game. Google 
AlphaGo program becomes the first Computer Go program to beat a professional human player . It  
is based on the  combination of machine lear